In [30]:
import random
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(user1, user2, records):
    user1_movies = records.loc[records['r_u_id'] == user1, 'r_content'].tolist()
    user2_movies = records.loc[records['r_u_id'] == user2, 'r_content'].tolist()

    common_movies = list(set(user1_movies) & set(user2_movies))

    if len(common_movies) == 0:
        return 0

    user1_vector = np.zeros(len(common_movies))
    user2_vector = np.zeros(len(common_movies))

    for i, movie in enumerate(common_movies):
        user1_vector[i] = int(movie in user1_movies)
        user2_vector[i] = int(movie in user2_movies)

    return cosine_similarity([user1_vector], [user2_vector])[0][0]

def collaborative_filtering(user, personality_table, record_table, movie_table, top_n=20):
    # 获取用户的偏好标签
    user_preferences = personality_table.loc[personality_table['p_u_id'] == int(user), ['p_label', 'p_label2', 'p_label3']].values.flatten()

    # 根据用户相似度推荐结果
    similar_users = personality_table['p_u_id'].unique().tolist()
    if user in similar_users:
        similar_users.remove(user)

    similarity_scores = []
    for similar_user in similar_users:
        similarity = calculate_similarity(user, similar_user, record_table)
        similarity_scores.append((similar_user, similarity))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    similar_recommendations = []

    for similar_user, similarity in similarity_scores:
        movies = record_table.loc[record_table['r_u_id'] == similar_user, 'r_content'].tolist()
        for movie in movies:
            if movie not in similar_recommendations:
                similar_recommendations.append(movie)
                if len(similar_recommendations) >= top_n:
                    break

        if len(similar_recommendations) >= top_n:
            break
    print(similar_recommendations)

    # 根据偏好标签推荐结果
    label_recommendations = []
    for preference in user_preferences:
        if pd.isnull(preference):
            continue

        movies = movie_table[movie_table['m_type'].str.contains(preference, na=False)]
        movies = movies.sort_values('m_score', ascending=False).head(top_n)['m_id'].tolist()
        label_recommendations.extend(movies)

    recommendations = similar_recommendations + label_recommendations
    # 去除推荐结果中的重复电影
    recommendations = list(set(recommendations))

    # 打乱最终推荐顺序
    random.shuffle(recommendations)

    return recommendations


# 连接数据库
def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="123456",
        database="film_class_web"
    )
    return connection


# 获取数据
def get_personality_table(connection):
    query = "SELECT * FROM personality_table"
    personality_table = pd.read_sql_query(query, connection)
    return personality_table


def get_record_table(connection):
    query = "SELECT * FROM record_table"
    record_table = pd.read_sql_query(query, connection)
    return record_table


def get_movie_table(connection):
    query = "SELECT * FROM movie_table"
    movie_table = pd.read_sql_query(query, connection)
    return movie_table


# 关闭数据库连接
def close_connection(connection):
    connection.close()


# 使用示例
def main():
    user_id = '2'  # 替换为您的用户ID

    # 连接到数据库
    connection = connect_to_database()

    # 获取数据表
    personality_table = get_personality_table(connection)
    record_table = get_record_table(connection)
    movie_table = get_movie_table(connection)

    # 执行协同过滤推荐
    recommendations = collaborative_filtering(user_id, personality_table, record_table, movie_table)

    print(recommendations)

    # 关闭数据库连接
    close_connection(connection)


if __name__ == "__main__":
    main()


['406', '465', '455', '497', '511', '411', '425', '423', '489', '532', '444', '501', '536', '437', '438', '435', '469', '452', '456', '471']
[443, 422, 436, 508, 416, '437', '511', 518, '411', 525, 512, 458, '425', 460, 529, '465', '423', 535, 441, '501', 488, 451, 452, 537, 521, 504, '444', 524, 412, '469', 421, '471', 540, '438', 523, 490, 447, 484, 506, 463, 538, 435, 414, 420, '489', 471, 464, 495, '497', 433, 472, 431, 510, '455', 509, 411, 445, 442, 489, 517, '406', 467, '532', 528, '452', '435', 461, '536', 526, 429, '456', 437]


C:\Users\DYL\AppData\Local\Temp\ipykernel_10092\3454844649.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  personality_table = pd.read_sql_query(query, connection)
C:\Users\DYL\AppData\Local\Temp\ipykernel_10092\3454844649.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  record_table = pd.read_sql_query(query, connection)
C:\Users\DYL\AppData\Local\Temp\ipykernel_10092\3454844649.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_table = pd.read_sql_query(query, connection)
